# 30. Data merge and split
## Contents
- Prerequisites
- Merge structured and unstructured data
- Train test (balanced) split
---------------------------------------------------------
## Prerequisites

In [2]:
import time
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from concurrent.futures import ThreadPoolExecutor
file_path = 'C:/Users/Koen.Janssen/OneDrive/Documents/TILBURG UNIVERSITY/Thesis/Data/'
parties = np.load('00_parties.npy')

------------------------------------------------
<br>
<br>
<br>


## Merge structured and unstructured data

In [15]:
dur = time.time()
# -------------------------------------------------------------------------------------~----------------------------------------


# step 1: load structured data
# -------------------------------------------------------------------------------------~----------------------------------------
print("\nstep 1: load structured data")                                                # print step
print("-----------------------------------------------------------------------")
file_url  = file_path + '12_data_structured_processed.csv'                             # define file url
data1 = pd.read_csv(file_url)                                                          # read structred data  
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(f"unique: {(not data1['stemming_id'].duplicated().any())}")                      # print duplicates
print(f"length: {len(data1)}")                                                         # print length
display(data1.head(1))                                                                 # display row
display((data1.isna().sum()).to_frame().T)                                             # display nulls
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



# step 2: load unstructured data
# -------------------------------------------------------------------------------------~----------------------------------------
print("\n\n\n step 2: load unstructured data")                                         # print step
print("-----------------------------------------------------------------------")
file_url  = file_path + '27_data_unstructured_refined.csv'                             # define file url
data2 = pd.read_csv(file_url)                                                          # read unstructred data
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(f"unique: {(not data2['document_id'].duplicated().any())}")                       # print duplicates
print(f"length: {len(data2)}")                                                          # print length
display(data2.head(1))                                                                  # display row
display((data2.isna().sum()).to_frame().T)                                              # display nulls
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


# step 3: merge data (structured and unstructured)
# -------------------------------------------------------------------------------------~----------------------------------------
print("\n\n\n step 3: merge data (structured and unstructured)")                       # print step
print("-----------------------------------------------------------------------")
data = data1.merge(data2, on='document_id')                                            # merge data
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(f"unique: {(not data['stemming_id'].duplicated().any())}")                       # print duplicates
print(f"length: {len(data)}")                                                          # print length
display(data.head(1))                                                                  # display row
display((data.isna().sum()).to_frame().T)                                              # display nulls
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



# -------------------------------------------------------------------------------------~----------------------------------------
file_url  = file_path + '31_data_merged.csv' 
data.to_csv(file_url, index=False)
print('\n---------------------------------------------------------------------------------------------------------------------')
print(f"Code duration: {round((time.time()  - dur),3)} seconds")   


step 1: load structured data
-----------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
unique: True
length: 301854


,stemming_id,document_id,party,day,vote
0,de29f06b-cab4-416b-82a1-cddc3a610551,08f70318-bcee-4284-95d8-25539822718c,50PLUS,7,1


,stemming_id,document_id,party,day,vote
0,0,0,0,0,0


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



 step 2: load unstructured data
-----------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
unique: True
length: 20666


,document_id,text,source,text_l,text_s,text_sl,text_fre,text_pol,text_sub
0,08f70318-bcee-4284-95d8-25539822718c,toestand rijks financiën europees regeldruk kl...,Nota over de toestand van ’s Rijks Financiën M...,-0.433714,-0.511112,-0.063503,0.437354,-1.154401,0.23354


,document_id,text,source,text_l,text_s,text_sl,text_fre,text_pol,text_sub
0,0,0,0,0,0,0,0,0,0


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



 step 3: merge data (structured and unstructured)
-----------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
unique: True
length: 301854


,stemming_id,document_id,party,day,vote,text,source,text_l,text_s,text_sl,text_fre,text_pol,text_sub
0,de29f06b-cab4-416b-82a1-cddc3a610551,08f70318-bcee-4284-95d8-25539822718c,50PLUS,7,1,toestand rijks financiën europees regeldruk kl...,Nota over de toestand van ’s Rijks Financiën M...,-0.433714,-0.511112,-0.063503,0.437354,-1.154401,0.23354


,stemming_id,document_id,party,day,vote,text,source,text_l,text_s,text_sl,text_fre,text_pol,text_sub
0,0,0,0,0,0,0,0,0,0,0,0,0,0


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

---------------------------------------------------------------------------------------------------------------------
Code duration: 26.32 seconds


In [8]:
dur = time.time()
# -------------------------------------------------------------------------------------~----------------------------------------


# step 1: load structured data
# -------------------------------------------------------------------------------------~----------------------------------------
print("\nstep 1: load structured data")                                                # print step
print("-----------------------------------------------------------------------")
file_url  = file_path + '31_data_merged.csv' 
data = pd.read_csv(file_url)                                                           # read structred data  

# step 2: identify documents where all parties have voted
# -------------------------------------------------------------------------------------~----------------------------------------
print("\nstep 2: identify documents where all parties have voted")                      # print step
print("-----------------------------------------------------------------------")
document_votes = data.groupby('document_id')['party'].nunique()                      
all_parties_voted_documents = document_votes[document_votes == len(parties)].index   


data = data[data['document_id'].isin(all_parties_voted_documents)]
data = data['document_id']
# Display the result
print(f"Number of documents where all parties have voted: {len(all_parties_voted_documents)}")
print("Sample data for documents where all parties have voted:")
display(data.head())

np.save('01_documents.npy', list(data))  # Save the list of common document IDs


step 1: load structured data
-----------------------------------------------------------------------

step 2: identify documents where all parties have voted
-----------------------------------------------------------------------
Number of documents where all parties have voted: 5428
Sample data for documents where all parties have voted:


53793    61259d62-aa61-4081-a3ad-22cfafa887f2
53794    61259d62-aa61-4081-a3ad-22cfafa887f2
53795    61259d62-aa61-4081-a3ad-22cfafa887f2
53796    61259d62-aa61-4081-a3ad-22cfafa887f2
53797    61259d62-aa61-4081-a3ad-22cfafa887f2
Name: document_id, dtype: object

------------------------------------------------
<br>
<br>
<br>


## Train test split

In [28]:
dur = time.time()
file_url = file_path + '31_data_merged.csv'
data = pd.read_csv(file_url)
# -------------------------------------------------------------------------------------~----------------------------------------
options = ['imbalanced', 'oversampling', 'undersampling', 'balancedsampling']          # define options (sampling methods)
combinations = [(party, option) for party in parties for option in options]            # define combinations of party and option
rs = 1                                                                                 # define random state

# step 1: source
# -------------------------------------------------------------------------------------~---------------------------------------
print("\n step 1: source")                                                             # print step
print("-----------------------------------------------------------------------")
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(f"unique: {(not data['stemming_id'].duplicated().any())}")                       # print duplicates
print(f"length: {len(data)}")                                                          # print length
display(data.head(1))                                                                  # display row
display((data.isna().sum()).to_frame().T)                                              # display nulls
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


# step 2: split
# -------------------------------------------------------------------------------------~---------------------------------------
print("\n step 2: split")                                                              # print step
print("-----------------------------------------------------------------------")
final_results = pd.DataFrame(columns=['party', 'option', 'totalrows', 'yesvotes', 'novotes'])

for party in parties:                                                                  # for each party
    for option in options:                                                             # for each option
        p_data = data[data['party'] == party]                                              # define party dataset
        p_data = p_data.drop('party', axis=1)                                              # redefine party dataset without party
        X = p_data.drop('vote', axis=1)                                                    # define X
        y = p_data['vote']                                                                 # define y

        # split data
        # -------------------------------------------------------------------------------------~------------------------------------
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=rs)
        X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=rs)
        X_trainvalid = pd.concat([X_train, X_valid])
        y_trainvalid = pd.concat([y_train, y_valid])    

        # imbalanced - no sampling
        # -------------------------------------------------------------------------------------~------------------------------------
        if option == 'imbalanced':
            # before TF-IDF & SVD (10)
            X_train.to_csv(file_path + f'10_models/imbalanced/{party}_X_train.csv', index=False)           
            X_valid.to_csv(file_path + f'10_models/imbalanced/{party}_X_valid.csv', index=False)                
            y_train.to_csv(file_path + f'10_models/imbalanced/{party}_y_train.csv', index=False)          
            y_valid.to_csv(file_path + f'10_models/imbalanced/{party}_y_valid.csv', index=False) 
            # after TF-IDF & SVD (20)
            y_train.to_csv(file_path + f'20_models/imbalanced/{party}_y_train.csv', index=False)          
            y_valid.to_csv(file_path + f'20_models/imbalanced/{party}_y_valid.csv', index=False)     

            # final model, before TF-IDF & SVD (30)  
            X_trainvalid.to_csv( file_path + f'30_models_imbalanced/{party}_X_trainvalid.csv', index=False)        
            X_test.to_csv(       file_path + f'30_models_imbalanced/{party}_X_test.csv'      , index=False)     
            y_trainvalid.to_csv( file_path + f'30_models_imbalanced/{party}_y_trainvalid.csv', index=False)           
            y_test.to_csv(       file_path + f'30_models_imbalanced/{party}_y_test.csv'      , index=False)
            # final model, after TF-IDF & SVD (40)  
            y_trainvalid.to_csv( file_path + f'40_models_imbalanced/{party}_y_trainvalid.csv', index=False)           
            y_test.to_csv(       file_path + f'40_models_imbalanced/{party}_y_test.csv'      , index=False)        

            y, n = y_train.sum(), len(y_train) - y_train.sum()



        # oversampling 
        # -------------------------------------------------------------------------------------~------------------------------------
        elif option == 'oversampling':
            oversampler = RandomOverSampler(sampling_strategy='auto', random_state=rs)
            X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

            # before TF-IDF & SVD (10)
            X_train_oversampled.to_csv( file_path + f'10_models/oversampling/{party}_X_train.csv', index=False)           
            X_valid.to_csv(             file_path + f'10_models/oversampling/{party}_X_valid.csv', index=False)                
            y_train_oversampled.to_csv( file_path + f'10_models/oversampling/{party}_y_train.csv', index=False)          
            y_valid.to_csv(             file_path + f'10_models/oversampling/{party}_y_valid.csv', index=False) 
            # after TF-IDF & SVD (20)
            y_train_oversampled.to_csv( file_path + f'20_models/oversampling/{party}_y_train.csv', index=False)          
            y_valid.to_csv(             file_path + f'20_models/oversampling/{party}_y_valid.csv', index=False)    

            y, n = y_train_oversampled.sum(), len(y_train_oversampled) - y_train_oversampled.sum()        



        # undersampling
        # -------------------------------------------------------------------------------------~------------------------------------
        elif option == 'undersampling':
            undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=rs)
            X_train_undersampled, y_train_undersampled = undersampler.fit_resample(X_train, y_train)

            # before TF-IDF & SVD (10)
            X_train_undersampled.to_csv( file_path + f'10_models/undersampling/{party}_X_train.csv', index=False)           
            X_valid.to_csv(              file_path + f'10_models/undersampling/{party}_X_valid.csv', index=False)                
            y_train_undersampled.to_csv( file_path + f'10_models/undersampling/{party}_y_train.csv', index=False)          
            y_valid.to_csv(              file_path + f'10_models/undersampling/{party}_y_valid.csv', index=False) 
            # after TF-IDF & SVD (20)
            y_train_undersampled.to_csv( file_path + f'20_models/undersampling/{party}_y_train.csv', index=False)          
            y_valid.to_csv(              file_path + f'20_models/undersampling/{party}_y_valid.csv', index=False)    

            y, n = y_train_undersampled.sum(), len(y_train_undersampled) - y_train_undersampled.sum()           


        # balanced sampling
        # -------------------------------------------------------------------------------------~------------------------------------
        elif option == 'balancedsampling':
            y, n = y_train.sum(), len(y_train) - y_train.sum()  
            samples = round(len(y_train) / 2)

            if y == n:
                # before TF-IDF & SVD (10)
                X_train.to_csv(file_path + f'10_models/balancedsampling/{party}_X_train.csv', index=False)           
                X_valid.to_csv(file_path + f'10_models/balancedsampling/{party}_X_valid.csv', index=False)                
                y_train.to_csv(file_path + f'10_models/balancedsampling/{party}_y_train.csv', index=False)          
                y_valid.to_csv(file_path + f'10_models/balancedsampling/{party}_y_valid.csv', index=False) 
                # after TF-IDF & SVD (20)
                y_train.to_csv(file_path + f'20_models/balancedsampling/{party}_y_train.csv', index=False)          
                y_valid.to_csv(file_path + f'20_models/balancedsampling/{party}_y_valid.csv', index=False) 

                y, n = y_train.sum(), len(y_train) - y_train.sum()  



            elif y > n:
                undersampler = RandomUnderSampler(sampling_strategy={1: samples}, random_state=rs)
                oversampler = RandomOverSampler(sampling_strategy={0: samples}, random_state=rs)
                X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)
                X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_resampled, y_train_resampled)\

                # before TF-IDF & SVD (10)
                X_train_resampled.to_csv( file_path + f'10_models/balancedsampling/{party}_X_train.csv', index=False)           
                X_valid.to_csv(           file_path + f'10_models/balancedsampling/{party}_X_valid.csv', index=False)                
                y_train_resampled.to_csv( file_path + f'10_models/balancedsampling/{party}_y_train.csv', index=False)          
                y_valid.to_csv(           file_path + f'10_models/balancedsampling/{party}_y_valid.csv', index=False) 
                # after TF-IDF & SVD (20)
                y_train_resampled.to_csv( file_path + f'20_models/balancedsampling/{party}_y_train.csv', index=False)          
                y_valid.to_csv(           file_path + f'20_models/balancedsampling/{party}_y_valid.csv', index=False)    

                y, n = y_train_resampled.sum(), len(y_train_resampled) - y_train_resampled.sum() 



            elif n > y:
                undersampler = RandomUnderSampler(sampling_strategy={0: samples}, random_state=rs)
                oversampler = RandomOverSampler(sampling_strategy={1: samples}, random_state=rs)
                X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)
                X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_resampled, y_train_resampled)

                # before TF-IDF & SVD (10)
                X_train_resampled.to_csv( file_path + f'10_models/balancedsampling/{party}_X_train.csv', index=False)           
                X_valid.to_csv(           file_path + f'10_models/balancedsampling/{party}_X_valid.csv', index=False)                
                y_train_resampled.to_csv( file_path + f'10_models/balancedsampling/{party}_y_train.csv', index=False)          
                y_valid.to_csv(           file_path + f'10_models/balancedsampling/{party}_y_valid.csv', index=False) 
                # after TF-IDF & SVD (20)
                y_train_resampled.to_csv( file_path + f'20_models/balancedsampling/{party}_y_train.csv', index=False)          
                y_valid.to_csv(           file_path + f'20_models/balancedsampling/{party}_y_valid.csv', index=False)   

                y, n = y_train_resampled.sum(), len(y_train_resampled) - y_train_resampled.sum() 

        # display results
        # -------------------------------------------------------------------------------------~------------------------------------
        result = pd.DataFrame({'party': [party], 'option': [option], 'totalrows': [y + n], 'yesvotes': [y], 'novotes': [n]})
        final_results = pd.concat([final_results, result], ignore_index=True)
        


display(final_results)                                                                 # display final results


# -------------------------------------------------------------------------------------~----------------------------------------
print('\n---------------------------------------------------------------------------------------------------------------------')
print(f"Code duration: {round((time.time()  - dur),3)} seconds")   


 step 1: source
-----------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
unique: True
length: 301854


,stemming_id,document_id,party,day,vote,text,source,text_l,text_s,text_sl,text_fre,text_pol,text_sub
0,de29f06b-cab4-416b-82a1-cddc3a610551,08f70318-bcee-4284-95d8-25539822718c,50PLUS,7,1,toestand rijks financiën europees regeldruk kl...,Nota over de toestand van ’s Rijks Financiën M...,-0.433714,-0.511112,-0.063503,0.437354,-1.154401,0.23354


,stemming_id,document_id,party,day,vote,text,source,text_l,text_s,text_sl,text_fre,text_pol,text_sub
0,0,0,0,0,0,0,0,0,0,0,0,0,0


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 step 2: split
-----------------------------------------------------------------------


,party,option,totalrows,yesvotes,novotes
0,50PLUS,imbalanced,12117,9578,2539
1,50PLUS,oversampling,19156,9578,9578
2,50PLUS,undersampling,5078,2539,2539
3,50PLUS,balancedsampling,12116,6058,6058
4,CDA,imbalanced,12383,6225,6158
...,...,...,...,...,...
67,NSC,balancedsampling,4246,2123,2123
68,Volt,imbalanced,5457,4130,1327
69,Volt,oversampling,8260,4130,4130
70,Volt,undersampling,2654,1327,1327



---------------------------------------------------------------------------------------------------------------------
Code duration: 73.115 seconds
